In [11]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [13]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [14]:
# 1. 프롬프트에 history 자리를 확보
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 도우미야, 간략하게 그냥 응답하도록 해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])
prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [15]:
chain = prompt | model | StrOutputParser()

In [16]:
# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id : str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

In [17]:
# 3. 히스토리랑 래핑
with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: get_store(session_id),
    input_messages_key="question",
    history_messages_key="history"
)

In [18]:
cfg = {"configurable" : {"session_id" : "user-123"}}
result = with_history.invoke({"question" : "내가 좋아 하는 폰은 갤럭시야"}, config= cfg)
print(result)

갤럭시 좋은 선택이야! 어떤 모델 좋아해?


In [19]:
result = with_history.invoke({"question" : "그 장점만 요약해봐"}, config= cfg)
print(result)

갤럭시 장점 요약:
- 뛰어난 디스플레이 품질 (AMOLED)
- 강력한 성능과 빠른 처리 속도
- 다양한 카메라 기능과 우수한 사진 품질
- 방수·방진 지원
- 확장 가능한 저장 공간
- 삼성 생태계와의 높은 호환성


In [20]:
result = with_history.invoke({"question" : "아이폰보다 좋아?"}, config= cfg)
print(result)

용도에 따라 다르지만, 갤럭시는 커스터마이징과 확장성, 디스플레이에서 강점이 있어. 아이폰은 보안과 최적화, 앱 생태계가 뛰어나.


In [21]:
result = with_history.invoke({"question" : "너의 개인적인 견해를 말해"}, config= cfg)
print(result)

나는 개인적인 견해가 없지만, 사용자 취향과 필요에 맞게 선택하는 게 가장 좋아.


In [22]:
stores["user-123"].messages

[HumanMessage(content='내가 좋아 하는 폰은 갤럭시야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 좋은 선택이야! 어떤 모델 좋아해?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='그 장점만 요약해봐', additional_kwargs={}, response_metadata={}),
 AIMessage(content='갤럭시 장점 요약:\n- 뛰어난 디스플레이 품질 (AMOLED)\n- 강력한 성능과 빠른 처리 속도\n- 다양한 카메라 기능과 우수한 사진 품질\n- 방수·방진 지원\n- 확장 가능한 저장 공간\n- 삼성 생태계와의 높은 호환성', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='아이폰보다 좋아?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='용도에 따라 다르지만, 갤럭시는 커스터마이징과 확장성, 디스플레이에서 강점이 있어. 아이폰은 보안과 최적화, 앱 생태계가 뛰어나.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너의 개인적인 견해를 말해', additional_kwargs={}, response_metadata={}),
 AIMessage(content='나는 개인적인 견해가 없지만, 사용자 취향과 필요에 맞게 선택하는 게 가장 좋아.', additional_kwargs={}, response_metadata={})]